In [ ]:
import pandas as pd
import numpy as np

# 1. Read the CSV files
math_df = pd.read_csv('MathScoreTerm1.csv')
ds_df = pd.read_csv('DSScoreTerm1.csv')
physics_df = pd.read_csv('PhysicsScoreTerm1.csv')

# Preview the data (optional)
# print(math_df.head())

# 2. Remove Name and Ethnicity columns to ensure confidentiality
# But we will keep Ethnicity temporarily if you want to do enhancement later
# For now, let's keep Ethnicity in a separate variable for enhancement
ethnicity_math = math_df['Ethnicity'].copy() if 'Ethnicity' in math_df.columns else None

cols_to_drop = ['Name', 'Ethnicity']
math_df = math_df.drop(columns=[col for col in cols_to_drop if col in math_df.columns])
ds_df = ds_df.drop(columns=[col for col in cols_to_drop if col in ds_df.columns])
physics_df = physics_df.drop(columns=[col for col in cols_to_drop if col in physics_df.columns])

# 3. Fill missing score data with zero
math_df = math_df.fillna(0)
ds_df = ds_df.fillna(0)
physics_df = physics_df.fillna(0)

# 4. Merge the three dataframes on Student ID or a common column
# Assuming there's a 'StudentID' or similar column to merge on. If not, we merge by index.
# Let's check columns to find a suitable key:

# Let's say 'StudentID' is the common key.
merged_df = math_df.merge(ds_df, on='StudentID', suffixes=('_Math', '_DS'))
merged_df = merged_df.merge(physics_df, on='StudentID')
# If 'StudentID' is not the key, use index-based merge instead:
# merged_df = math_df.merge(ds_df, left_index=True, right_index=True)
# merged_df = merged_df.merge(physics_df, left_index=True, right_index=True)

# 5. Convert Sex column to numerical: M=1, F=2
# Assuming column is named 'Sex' in merged_df
if 'Sex' in merged_df.columns:
    merged_df['Sex'] = merged_df['Sex'].map({'M': 1, 'F': 2})

# 6. Store the cleaned data into a new CSV
merged_df.to_csv('ScoreFinal.csv', index=False)

# --- ENHANCEMENTS ---

# Enhancement 1: Convert Ethnicity to numerical value (if you kept ethnicity in a separate variable)
# Using Label Encoding

if ethnicity_math is not None:
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    ethnicity_encoded = le.fit_transform(ethnicity_math)
    # Add it back to merged_df before dropping ethnicity originally:
    merged_df['Ethnicity'] = ethnicity_encoded
    # Save again with ethnicity encoded
    merged_df.to_csv('ScoreFinal_withEthnicity.csv', index=False)

# Enhancement 2: Instead of filling missing scores with zero, fill with average score of the class
# Let's reload original data to apply this approach:

def fill_missing_with_class_avg(df, score_cols):
    for col in score_cols:
        avg_score = df[col].mean()
        df[col].fillna(avg_score, inplace=True)
    return df

# Assuming score columns names as 'Score' or specific ones, let's try:
# For example, columns might be ['MathScore', 'DSScore', 'PhysicsScore']

# Since each file had only one score column, let's check and fill accordingly:
math_df = pd.read_csv('MathScoreTerm1.csv').drop(columns=[col for col in cols_to_drop if col in math_df.columns])
ds_df = pd.read_csv('DSScoreTerm1.csv').drop(columns=[col for col in cols_to_drop if col in ds_df.columns])
physics_df = pd.read_csv('PhysicsScoreTerm1.csv').drop(columns=[col for col in cols_to_drop if col in physics_df.columns])

math_df = fill_missing_with_class_avg(math_df, math_df.columns.difference(['StudentID', 'Sex']))
ds_df = fill_missing_with_class_avg(ds_df, ds_df.columns.difference(['StudentID', 'Sex']))
physics_df = fill_missing_with_class_avg(physics_df, physics_df.columns.difference(['StudentID', 'Sex']))

# Merge again
merged_df_avg = math_df.merge(ds_df, on='StudentID', suffixes=('_Math', '_DS'))
merged_df_avg = merged_df_avg.merge(physics_df, on='StudentID')

# Convert Sex again
if 'Sex' in merged_df_avg.columns:
    merged_df_avg['Sex'] = merged_df_avg['Sex'].map({'M': 1, 'F': 2})

merged_df_avg.to_csv('ScoreFinal_AvgImputed.csv', index=False)
